In [ ]:
import tensorflow as tf

stringi = tf.Variable("this is a srring", tf.string)
number = tf.Variable(324, tf.int16)
floating = tf.Variable(3.5689, tf.float64)

rank3_tensor = tf.Variable([[["hello", "ad hi"]], [["hello", "ad hi"]]])
# tf.rank(rank3_tensor)
# rank3_tensor.shape

tensor1 = tf.ones([2, 3])
tensor2 = tf.reshape(tensor1, [3, 2])
tensor3 = tf.transpose(tensor2)
print(tensor3)

In [ ]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import, division, print_function
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.compat.v2.feature_column as fc
from six.moves import urllib

dftrain =pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dfeval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

# dftain.head()
# dftrain.shape
# dftrain.descibe()
# y_train.head()
# dftrain.age.hist(bins=20)
# dftrain.sex.value_counts().plot(kind='barh')
# dftrain['class'].value_counts().plot(kind='barh')

pd.concat([dftrain, y_train], axis =1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
feature_column = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique() # unique values will be stored in vocabulary
  feature_column.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_column.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
print(feature_column)

: 

In [ ]:
import tensorflow_probability as tfp
import tensorflow as tf

tfd = tfp.distributions
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])
transition_distributin = tfd.Categorical(probs=[[0.7, 0.3], [0.2, 0.8]])
observation_distribution = tfd.Normal(loc = [0., 15.], scale = [5., 10.])
model = tfd.HiddenMarkovModel(initial_distribution = initial_distribution,
                              transition_distribution = transition_distributin,
                              observation_distribution = observation_distribution,
                              num_steps = 7)
mean = model.mean()
with tf.compat.v1.Session() as sess:
  print(mean.numpy())

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# train.head()
# train.tail()
train_y = train.pop('Species')
test_y = test.pop('Species')
# train.shape
def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30, 10], n_classes=3)
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000)
eval_results = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set Accuracy: {accuracy:0.3f}\n'.format(**eval_results))
